<a href="https://colab.research.google.com/github/zihoonkim/inf_car_number/blob/main/cnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


import tensorflow as tf


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data() ## minst가 아닌 cifar10 데이터를 불러온 뒤 load
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train / 255., x_test / 255.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)  # squeeze [1,32,32,3] > [32,32,3]

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train_one_hot))
train_data = train_data.repeat().shuffle(50000).batch(128)
train_data_iter = iter(train_data)


test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test_one_hot))
test_data = test_data.batch(1000) ## 테스트 데이터의 양도 많기 때문에 test data도 1000개씩 배치로 묶어 진행
test_data_iter = iter(test_data)

# CNN 모델을 정의
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    # 첫번째 convolutional layer .
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2)
    # 두번째 convolutional layer 
    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2)
    # 세번째 convolutional layer
    self.conv_layer_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu')
    # 네번째 convolutional layer
    self.conv_layer_4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu')
    # 다섯번째 convolutional layer
    self.conv_layer_5 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu')
  
    
    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(384, activation='relu')
    self.dropout = tf.keras.layers.Dropout(0.2) # 20%의 drop out을 실행

    
    self.output_layer = tf.keras.layers.Dense(10, activation=None)

  def call(self, x, is_training):
    # 입력 이미지
    h_conv1 = self.conv_layer_1(x)
    h_pool1 = self.pool_layer_1(h_conv1)
    h_conv2 = self.conv_layer_2(h_pool1)
    h_pool2 = self.pool_layer_2(h_conv2)
    h_conv3 = self.conv_layer_3(h_pool2)
    h_conv4 = self.conv_layer_4(h_conv3)
    h_conv5 = self.conv_layer_5(h_conv4)
    h_conv5_flat = self.flatten_layer(h_conv5)
    h_fc1 = self.fc_layer_1(h_conv5_flat)
    # Dropout 
    h_fc1_drop = self.dropout(h_fc1, training=is_training) # is_training 이 yes 면 드롭아웃 진행 
    logits = self.output_layer(h_fc1_drop)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

# cross-entropy 손실 함수를 정의
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화
optimizer = tf.optimizers.RMSprop(1e-3)

# 최적화를 위한 function을 정의
@tf.function
def train_step(model, x, y, is_training):
  with tf.GradientTape() as tape:
    y_pred, logits = model(x, is_training)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# 모델의 정확도를 출력하는 함수를 정의
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy


CNN_model = CNN()


for i in range(10000):
  batch_x, batch_y = next(train_data_iter)

 
  if i % 100 == 0:
    train_accuracy = compute_accuracy(CNN_model(batch_x, False)[0], batch_y)
    loss_print = cross_entropy_loss(CNN_model(batch_x, False)[1], batch_y)

    print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))

  train_step(CNN_model, batch_x, batch_y, True)

# 학습이 끝나면 테스트 데이터(10000개)에 대한 정확도를 출력합니다.
test_accuracy = 0.0
for i in range(10):
  test_batch_x, test_batch_y = next(test_data_iter)
  test_accuracy = test_accuracy + compute_accuracy(CNN_model(test_batch_x, False)[0], test_batch_y).numpy()
test_accuracy = test_accuracy / 10
print("테스트 데이터 정확도: %f" % test_accuracy)

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.070312, 손실 함수(loss): 2.301987
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.242188, 손실 함수(loss): 2.127976
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.273438, 손실 함수(loss): 1.959370
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.343750, 손실 함수(loss): 1.921126
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.460938, 손실 함수(loss): 1.372730
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.539062, 손실 함수(loss): 1.419447
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.593750, 손실 함수(loss): 1.277486
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.562500, 손실 함수(loss): 1.369476
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.460938, 손실 함수(loss): 1.382659
반복(Epoch): 900, 트레이닝 데이터 정확도: 0.593750, 손실 함수(loss): 1.256733
반복(Epoch): 1000, 트레이닝 데이터 정확도: 0.617188, 손실 함수(loss): 1.094352
반복(Epoch): 1100, 트레이닝 데이터 정확도: 0.601562, 손실 함수(loss): 1.113200
반복(Epoch): 1200, 트레이닝 데이터 정확도: 0.609375, 손실 함수(loss): 1.115607
반복(Epoch): 1300, 트레이닝 데이터 정확도: 0.648438, 손실 함수(loss): 0.989587
반복(Epoch): 1400, 트레이닝 데이터 정확도: 0.617188, 손실 함수(loss): 1.136505
반복(Epoch): 1500, 트레이닝 데이터 정확도: 0.726562, 손실 함수(loss): 0.764191
반복(E